In [1]:
import os
import torch
from torchvision import datasets, transforms
from torchvision.models import mobilenet_v3_large
from PIL import Image

import torchvision.transforms as transforms
import torch.nn as nn

In [ ]:

cuda_available = torch.cuda.is_available()
device = torch.device("cuda" if cuda_available else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [2]:


# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    "train": transforms.Compose(
        [
            transforms.RandomResizedCrop(
                224
            ),  # ImageNet models were trained on 224x224 images
            transforms.RandomHorizontalFlip(),  # flip horizontally 50% of the time - increases train set variability
            transforms.ToTensor(),  # convert it to a PyTorch tensor
            transforms.Normalize(
                [0.485, 0.456, 0.406], [0.229, 0.224, 0.225]
            ),  # ImageNet models expect this norm
        ]
    ),
    "val": transforms.Compose(
        [
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        ]
    ),
}

data_dir = "dataset"
# Create train and validation datasets and loaders
image_datasets = {
    x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x])
    for x in ["train", "val"]
}
dataloaders = {
    x: torch.utils.data.DataLoader(
        image_datasets[x], batch_size=4, shuffle=True, num_workers=0
    )
    for x in ["train", "val"]
}
dataset_sizes = {x: len(image_datasets[x]) for x in ["train", "val"]}
class_names = image_datasets["train"].classes
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:


# Load the model
model = mobilenet_v3_large(pretrained=False)
model.classifier[3] = nn.Linear(1280, 5)  # Assuming ImageNet classes, adjust if needed
model.load_state_dict(torch.load('models/MobileNetV3_large_transfer_final.pth'), strict=False)
model.eval()

# Preprocess the image
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

image = Image.open('ro2.jpg')
input_tensor = transform(image).unsqueeze(0)

# Perform inference
with torch.no_grad():
    outputs = model(input_tensor)
    _, predicted = torch.max(outputs, 1)


    
probabilities = torch.softmax(outputs, dim=1)
sorted_probs, sorted_indices = torch.sort(probabilities, descending=True)
for i in range(len(class_names)):
    print(f"{class_names[sorted_indices[0][i]]}: {sorted_probs[0][i].item():.4f}")

c:\Users\romai\Desktop\Prog\portable_nn_reckonizer\.venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\romai\Desktop\Prog\portable_nn_reckonizer\.venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


margot_robbie: 0.3712
ryan_gosling: 0.2851
gollum: 0.1629
romain: 0.0928
voldemort: 0.0879
